**Word level Multilayer LSTM Model**

---




In [10]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
!pip install --upgrade treebank
!pip install nltk
import treebank
import nltk
nltk.download("punkt")
nltk.download("stopwords")
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
#import libraries for model building
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense,Embedding,Dropout,GaussianNoise
import keras
# import 

**Take subset of whole dataset and removing specific stopwords to have more meaningful data**

In [3]:
# Train_data
data = treebank.penn['train']
data_train = data[:500000]
# stopwords.words('english')
stop_words = set()
stop_words.add('unk')

word_tokens = word_tokenize(data_train)

filtered_sentence = [w for w in word_tokens if not w in stop_words]

# print(word_tokens[10:50])
# print(filtered_sentence[10:50])
data_f = " "
data_final = data_f.join(filtered_sentence)
print(type(data_final))

<class 'str'>


**Preprocessing Test data**

In [4]:
#test data
print(len(treebank.penn['test']))
d_Test= treebank.penn['test']
d_Test = d_Test[:10000]

word_tokens_test = word_tokenize(d_Test)
stop_words = set()
stop_words.add('unk')
filtered_sentence_test = [w for w in word_tokens_test if not w in stop_words]
 
# print(word_tokens_test[10:50])
# print(filtered_sentence_test[10:50])

data_f_test = " "
data_final_test = data_f_test.join(filtered_sentence_test)

449945


**Use Tokenizer to remove punctuations and encode words into a numbers and assign it in a dictionary** 


In [5]:
# Encoding on Train Data
tokenizer=Tokenizer(num_words=None,filters='#$%&"()*+-<=>@[\\]^_`{|}~\t\n',lower = True, split = ' ')
# tokenizer.fit_on_texts([data_final])
train_tokens=tokenizer.fit_on_texts([data_final,data_final_test])

# convert text into sequences of numbers
encoded=tokenizer.texts_to_sequences([data_final])[0]

#define total vocabulary size
vocab_size=len(tokenizer.word_index)+1

#create seperate lists of 2 sequences since we will use 1 of them as input and one as output
sequences=[]
num_features = 5
for i in range(len(encoded)-(num_features+1)):
  sequences.append(encoded[i:i+(num_features+1)])
# len(sequences)
# sequences[:10]

#create dependent and independent variable
x=[]
y=[]
# x = sequences[:,0]
# y = sequences[:,1]
for i in range(len(sequences)):
    row = []
    [row.append(sequences[i][j]) for j in range(num_features)]
    x.append(row)
    y.append(sequences[i][num_features])

#convert list to an array
x=np.array(x)
y=np.array(y)
# print(x.shape)
# print(y.shape)
y=to_categorical(y,vocab_size)

In [6]:
# Encoding on Test Data
encoded_test=tokenizer.texts_to_sequences([data_final_test])[0]

#create seperate lists of 2 sequences since we will use 1 of them as input and one as output
sequences_test = []
num_features = 5
for i in range(len(encoded_test)-(num_features+1)):
    sequences_test.append(encoded_test[i:i+(num_features+1)])
    
#create dependent and independent variable
x_test = []
y_test = []

 

for i in range(len(sequences_test)):
    row = []
    [row.append(sequences_test[i][j]) for j in range(num_features)]
    x_test.append(row)
    y_test.append(sequences_test[i][num_features])

 

#convert list to an array
x_test = np.array(x_test)
y_test = np.array(y_test)

 

y_test = to_categorical(y_test,vocab_size)
# print(x_test.shape)
# print(y_test.shape)

**Build Model with optimizer = adam, Dropout  = 20%**

In [ ]:
#start building model
model=Sequential()
#add Embedding layer
model.add(Embedding(input_dim=vocab_size,output_dim=100,input_length=num_features))
#add LSTM layer
model.add(LSTM(64,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64))
#output layer
model.add(Dense(vocab_size,activation='softmax'))

**Compile and fit model to train on dataset**

In [ ]:
#compile 
model.compile('adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 5, 100)            659500    
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 64)             42240     
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 64)             0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_1 (Dense)              (None, 6595)              428675    
Total params: 1,163,439
Trainable params: 1,163,439
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training model
# history = model.fit(x,y,batch_size=1,epochs=10,validation_data=(x_test,y_test))
history = model.fit(x,y,batch_size=1,epochs=20,validation_split =0.2)

Epoch 1/20
65494/65494 [==============================] - 838s 13ms/step - loss: 7.3514 - accuracy: 0.0716 - val_loss: 7.7389 - val_accuracy: 0.1090
Epoch 2/20
65494/65494 [==============================] - 838s 13ms/step - loss: 7.2570 - accuracy: 0.1124 - val_loss: 7.6762 - val_accuracy: 0.1246
Epoch 3/20
65494/65494 [==============================] - 832s 13ms/step - loss: 6.8494 - accuracy: 0.1335 - val_loss: 7.7472 - val_accuracy: 0.1366
Epoch 4/20
65494/65494 [==============================] - 831s 13ms/step - loss: 6.4923 - accuracy: 0.1505 - val_loss: 8.0338 - val_accuracy: 0.1353
Epoch 5/20
65494/65494 [==============================] - 823s 13ms/step - loss: 6.2060 - accuracy: 0.1642 - val_loss: 8.1643 - val_accuracy: 0.1314
Epoch 6/20
65494/65494 [==============================] - 824s 13ms/step - loss: 5.9608 - accuracy: 0.1737 - val_loss: 8.3076 - val_accuracy: 0.1383
Epoch 7/20
65494/65494 [==============================] - 822s 13ms/step - loss: 5.6753 - accuracy: 0.1842

**Accuracy based on 1st model**

In [ ]:
oss_tr, accuracy_tr = model.evaluate(x, y, verbose=False)
print("Training Accuracy:  {:.4f}".format(accuracy_tr)) 

loss, accuracy = model.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))  

Training Accuracy:  0.3252
Testing Accuracy:  0.1159


**2nd Model using optimizer = SGD and Regularization with std deviation of 0.075**

In [7]:
mlstm=Sequential()
#add Embedding layer
mlstm.add(Embedding(input_dim=vocab_size,output_dim=100,input_length=num_features))
#add LSTM layer
mlstm.add(LSTM(64,return_sequences=True))
mlstm.add(GaussianNoise(0.075))
mlstm.add(LSTM(64))
mlstm.add(GaussianNoise(0.075))
#output layer
mlstm.add(Dense(vocab_size,activation='softmax'))

In [11]:
mlstm.compile(optimizer=keras.optimizers.SGD(lr=0.0001,momentum=0.99),
 loss='categorical_crossentropy',
 metrics=['accuracy'])
mlstm.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 5, 100)            659500    
_________________________________________________________________
lstm (LSTM)                  (None, 5, 64)             42240     
_________________________________________________________________
gaussian_noise (GaussianNois (None, 5, 64)             0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 6595)              428675    
Total params: 1,163,439
Trainable params: 1,163,439
Non-trainable params: 0
______________________________________________

In [12]:
history2 = mlstm.fit(x,y,batch_size=1,epochs=20,validation_split =0.2)

Epoch 1/20
65494/65494 [==============================] - 333s 5ms/step - loss: 7.3910 - accuracy: 0.0521 - val_loss: 6.7982 - val_accuracy: 0.0601
Epoch 2/20
65494/65494 [==============================] - 334s 5ms/step - loss: 6.7760 - accuracy: 0.0560 - val_loss: 6.7952 - val_accuracy: 0.0601
Epoch 3/20
65494/65494 [==============================] - 340s 5ms/step - loss: 6.7275 - accuracy: 0.0613 - val_loss: 6.7499 - val_accuracy: 0.0728
Epoch 4/20
65494/65494 [==============================] - 333s 5ms/step - loss: 6.6765 - accuracy: 0.0686 - val_loss: 6.6764 - val_accuracy: 0.0743
Epoch 5/20
65494/65494 [==============================] - 335s 5ms/step - loss: 6.5771 - accuracy: 0.0750 - val_loss: 6.5759 - val_accuracy: 0.0840
Epoch 6/20
65494/65494 [==============================] - 347s 5ms/step - loss: 6.4806 - accuracy: 0.0808 - val_loss: 6.5243 - val_accuracy: 0.0859
Epoch 7/20
65494/65494 [==============================] - 356s 5ms/step - loss: 6.3959 - accuracy: 0.0874 - val_

In [13]:
oss_tr, accuracy_tr = mlstm.evaluate(x, y, verbose=False)
print("Training Accuracy:  {:.4f}".format(accuracy_tr)) 

loss, accuracy = mlstm.evaluate(x_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))  

Training Accuracy:  0.1597
Testing Accuracy:  0.1311


In [14]:
#testing function
def predict_next_word(model,tokenizer,text,n_words, n_features):
    result=''
    out_word=[]
   
    for i in range(n_words):
        encoded_data_list = []
        for i in range(n_features):
            encoded_data=tokenizer.texts_to_sequences([text[i]])[0]           
            encoded_data_list.append(encoded_data[0])
           
        encoded_data_list=np.array(encoded_data_list)
        encoded_data_list=encoded_data_list.reshape(1,n_features)
        output=np.argmax(model.predict(encoded_data_list))
       
       
        for index,word in tokenizer.word_index.items():
            if word==output:
                out_word.append(index)
                text.append(index)
        text = text[1:]
    result=' '.join(out_word)
    return result

In [16]:
#testing 
# predict_next_word(model,tokenizer,['jones', 'friday', 'industrial', 'average', 'stock'],20,num_features)

**Predicting output based on the trained model**

In [17]:
predict_next_word(mlstm,tokenizer,['jones', 'friday', 'industrial', 'average', 'stock'],10,num_features)

'index futures trading and chief executive officer of the u.s.'